In [5]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("example-app") \
    .getOrCreate()

sc = spark.sparkContext

In [6]:
# brake train

import pandas as pd
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
import locale
import math
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

df = pd.read_csv('https://raw.githubusercontent.com/werowe/mist_preventive_maintenance_ml/master/brakedata.csv', sep= ',')
 
brakeData =  df.iloc[:,0:3]

a = [] 


def parsePoint(w,k,h):
    return LabeledPoint(worn, [km, heat])

for row in brakeData.itertuples():
	worn = getattr(row, 'worn')
	km = locale.atof(getattr(row, 'km'))
	heat = getattr(row,'heat')
	lp = parsePoint (worn, km, heat)
	a.append(lp)


lrm = LogisticRegressionWithLBFGS.train(sc.parallelize(a))

lrm.save(sc, "/tmp/brakeModel")

p = sc.parallelize(a)

valuesAndPreds = p.map(lambda p: (p.label, lrm.predict(p.features)))
 

accurate = 1 - valuesAndPreds.map(lambda vp: math.fabs(vp[0] - vp[1])).reduce(lambda x, y: x + y) / valuesAndPreds.count()

24/04/30 10:22:33 WARN Instrumentation: [5e44161d] Initial coefficients will be ignored! Its dimensions (1, 2) did not match the expected size (1, 2)
24/04/30 10:22:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/30 10:22:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [8]:


for value in valuesAndPreds.collect():
    print(value)

(1.0, 1)
(0.0, 0)
(1.0, 1)
(0.0, 0)
(1.0, 1)
(1.0, 1)
(1.0, 1)
(0.0, 1)
(0.0, 0)
(0.0, 0)
(1.0, 1)
(1.0, 1)


this part we need to modify to run on spark it was designed to run on mist

In [ ]:
from mist.mist_job import MistJob
import os
import numpy as np
import pandas as pd
import math
from numpy import array
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionModel

class Predict(MistJob):
 
    def do_stuff(self, parameters):
        val = parameters.values()
        list = val.head()
        size = list.size()
        pylist = []
        count = 0
        while count < size:
            pylist.append(list.head())
            count = count + 1
            list = list.tail()


        heat = pylist[0]
        km = pylist[1]
        lrm = LogisticRegressionModel.load(self.context, "/tmp/brakeModel")
        worn = lrm.predict([km,heat])
        return ("brake is worn=", worn)